In [7]:

from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq 
from langchain.agents import Tool , initialize_agent, AgentType
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from tavily import TavilyClient
import os
from dotenv import load_dotenv
load_dotenv()
tavily_key = os.getenv("TAVILY_API_KEY")



In [8]:

gpt35 = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.7,
    api_key=os.environ.get("OPENAI_API_KEY"),
)

llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",   ##meta-llama/llama-4-scout-17b-16e-instruct     llama-3.3-70b-versatile   deepseek-r1-distill-llama-70b
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [9]:
FORMATO_CURSOS = """
1. Curso: Nome do Curso
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
"""

In [10]:

tavily = TavilyClient(api_key=tavily_key)

PUBLICAS_KEYWORDS = [
    "uf", "usp", "unicamp", "ufmg", "ufrj", "ufpe", "ufsc", "ufabc", "unb", "if"
]

def func_buscar_cursos(query: str) -> list:
    resultados = tavily.search(query, search_depth="advanced", max_results=30)
    cursos = []

    for item in resultados["results"]:
        url_lower = item["url"].lower()
        titulo_lower = item["title"].lower()
        
        # Filtra só universidades públicas
        if any(pub in url_lower or pub in titulo_lower for pub in PUBLICAS_KEYWORDS):
            cursos.append({
                "titulo": item["title"],
                "url": item["url"],
                "conteudo": item["content"][:600]
            })
        if len(cursos) >= 5:  
            break
    return cursos




buscar_cursos = Tool(
    name="BuscadorCursos",
    func=func_buscar_cursos,
    description="Busca cursos superior na internet ou base de dados publica"
)

In [11]:
# template para estruturar perfil com palavras-chave + cursos


prompt_educacao = ChatPromptTemplate.from_messages([
    ("system", 
        "Você é especialista em educação brasileira. "
        "Sua resposta deve estar estritamente neste formato:"
        f"{FORMATO_CURSOS}\n"
        "- Gere sempre 3 cursos.\n"
        "- Cada curso com 5 universidades listada\n"
        "- **Não inclua nenhuma universidade ou centro privado ou site de conteúdo educacional.**"
        "- Use dados reais, não invente URLs.\n"
        "- Não use JSON nem outro formato de código."
    ),
    ("user", "Perfil: {perfil_usuario}")
])





In [12]:


chain_educacao = LLMChain(
    llm=llm,
    prompt=prompt_educacao,
    verbose=True
    
)

agente_pesquisador = initialize_agent(
    tools=[buscar_cursos],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,   ##STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION   ZERO_SHOT_REACT_DESCRIPTION  
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5,             
    early_stopping_method="generate" 
)



def orquestrador(perfil_usuario: str):
    sugestoes = chain_educacao.run({"perfil_usuario": perfil_usuario})

    promptpesquisador = (
        f"Com base nestas sugestões: {sugestoes}, "
        "encontre os cursos mais relevantes em instituições e universidades públicas, "
        f"e escreva tudo exatamente assim:\n{FORMATO_CURSOS} listando os 3 cursos e as universidades, - **Não inclua nenhuma universidade ou centro privado ou site de conteúdo educacional.**"
    )

    return agente_pesquisador.run(promptpesquisador)


perfil = "sou do integrado de edificações, gosto da parte de arquitetura urbanismo e designer de interiores, gosto de arte em geral, teatros e musicas,sou extrovertido no geral"

resposta = orquestrador(perfil)










> Entering new LLMChain chain...
Prompt after formatting:
System: Você é especialista em educação brasileira. Sua resposta deve estar estritamente neste formato:
1. Curso: Nome do Curso
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)
   - Universidade : Nome da Universidade (link: https://exemplo.com)

- Gere sempre 3 cursos.
- Cada curso com 5 universidades listada
- **Não inclua nenhuma universidade ou centro privado ou site de conteúdo educacional.**- Use dados reais, não invente URLs.
- Não use JSON nem outro formato de código.
Human: Perfil: sou do integrado de edificações, gosto da parte de arquitetura urbanismo e designer de interiores, gosto de arte em geral, teatros e musicas,sou extrovertido no geral

> Finished chain.


> Entering new AgentExecutor 